In [19]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from tensorflow.keras.models import load_model  # Use TensorFlow version of Keras
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server

# Load and preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
#becareful about csv file path
df_nse = pd.read_csv("./NSE-Tata-Global-Beverages-Limited.csv")
df_nse["Date"] = pd.to_datetime(df_nse["Date"], format="%Y-%m-%d")
df_nse.set_index('Date', inplace=True)  # Set 'Date' as the index

# Sort the data by index
data = df_nse.sort_index(ascending=True)

# Create new DataFrame for training and validation
new_data = pd.DataFrame(data['Close'])  # Directly using the 'Close' prices
dataset = new_data.values

# Split the dataset into train and valid
train = dataset[:987, :]
valid = dataset[987:, :]

# Scale the data
scaled_data = scaler.fit_transform(dataset)

# Prepare training data
x_train, y_train = [], []
for i in range(60, len(train)):
    x_train.append(scaled_data[i-60:i, 0])
    y_train.append(scaled_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Load the model
model = load_model("saved_model.h5")

# Prepare input for prediction
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1, 1)
inputs = scaler.transform(inputs)

# Prepare testing data for predictions
X_test = []
for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Predicting closing price
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

# Prepare the valid dataset with predictions
valid_df = new_data.iloc[987:].copy()  # Use .copy() to avoid SettingWithCopyWarning
valid_df['Predictions'] = closing_price

# Load additional data for stocks
df = pd.read_csv("./stock_data.csv")

# Define layout of the Dash app
app.layout = html.Div([
    html.H1("Stock Price Analysis Dashboard", style={"textAlign": "center"}),
    dcc.Tabs(id="tabs", children=[
        dcc.Tab(label='NSE-TATAGLOBAL Stock Data', children=[
            html.Div([
                html.H2("Actual Closing Price", style={"textAlign": "center"}),
                dcc.Graph(
                    id="Actual Data",
                    figure={
                        "data": [
                            go.Scatter(
                                x=valid_df.index,
                                y=valid_df["Close"],
                                mode='lines',  # Changed to lines for better visualization
                                name='Actual Close Price'
                            )
                        ],
                        "layout": go.Layout(
                            title='Actual Closing Prices',
                            xaxis={'title': 'Date'},
                            yaxis={'title': 'Closing Rate'}
                        )
                    }
                ),
                html.H2("LSTM Predicted Closing Price", style={"textAlign": "center"}),
                dcc.Graph(
                    id="Predicted Data",
                    figure={
                        "data": [
                            go.Scatter(
                                x=valid_df.index,
                                y=valid_df["Predictions"],
                                mode='lines',  # Changed to lines for consistency
                                name='Predicted Close Price'
                            )
                        ],
                        "layout": go.Layout(
                            title='Predicted Closing Prices',
                            xaxis={'title': 'Date'},
                            yaxis={'title': 'Closing Rate'}
                        )
                    }
                )
            ])
        ]),
        dcc.Tab(label='Facebook Stock Data', children=[
            html.Div([
                html.H1("Facebook Stocks High vs Lows", style={'textAlign': 'center'}),
                dcc.Dropdown(
                    id='my-dropdown',
                    options=[
                        {'label': 'Tesla', 'value': 'TSLA'},
                        {'label': 'Apple', 'value': 'AAPL'}, 
                        {'label': 'Facebook', 'value': 'FB'}, 
                        {'label': 'Microsoft', 'value': 'MSFT'}
                    ], 
                    multi=True,
                    value=['FB'],
                    style={"display": "block", "margin-left": "auto", "margin-right": "auto", "width": "60%"}
                ),
                dcc.Graph(id='highlow'),
                html.H1("Facebook Market Volume", style={'textAlign': 'center'}),
                dcc.Dropdown(
                    id='my-dropdown2',
                    options=[
                        {'label': 'Tesla', 'value': 'TSLA'},
                        {'label': 'Apple', 'value': 'AAPL'}, 
                        {'label': 'Facebook', 'value': 'FB'},
                        {'label': 'Microsoft', 'value': 'MSFT'}
                    ], 
                    multi=True,
                    value=['FB'],
                    style={"display": "block", "margin-left": "auto", "margin-right": "auto", "width": "60%"}
                ),
                dcc.Graph(id='volume')
            ], className="container"),
        ])
    ])
])

@app.callback(Output('highlow', 'figure'), [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown):
    dropdown = {"TSLA": "Tesla", "AAPL": "Apple", "FB": "Facebook", "MSFT": "Microsoft"}
    trace1, trace2 = [], []
    
    for stock in selected_dropdown:
        trace1.append(
            go.Scatter(
                x=df[df["Stock"] == stock]["Date"],
                y=df[df["Stock"] == stock]["High"],
                mode='lines', 
                opacity=0.7, 
                name=f'High {dropdown[stock]}', 
                textposition='bottom center'
            )
        )
        trace2.append(
            go.Scatter(
                x=df[df["Stock"] == stock]["Date"],
                y=df[df["Stock"] == stock]["Low"],
                mode='lines', 
                opacity=0.6,
                name=f'Low {dropdown[stock]}', 
                textposition='bottom center'
            )
        )
    
    data = trace1 + trace2
    figure = {
        'data': data,
        'layout': go.Layout(
            colorway=["#5E0DAC", '#FF4F00', '#375CB1', '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"High and Low Prices for {', '.join(dropdown[i] for i in selected_dropdown)} Over Time",
            xaxis={
                "title": "Date",
                'rangeselector': {
                    'buttons': [
                        {'count': 1, 'label': '1M', 'step': 'month', 'stepmode': 'backward'},
                        {'count': 6, 'label': '6M', 'step': 'month', 'stepmode': 'backward'},
                        {'step': 'all'}
                    ]
                },
                'rangeslider': {'visible': True}, 
                'type': 'date'
            },
            yaxis={"title": "Price (USD)"}
        )
    }
    return figure

@app.callback(Output('volume', 'figure'), [Input('my-dropdown2', 'value')])
def update_volume_graph(selected_dropdown_value):
    dropdown = {"TSLA": "Tesla", "AAPL": "Apple", "FB": "Facebook", "MSFT": "Microsoft"}
    trace1 = []
    
    for stock in selected_dropdown_value:
        trace1.append(
            go.Scatter(
                x=df[df["Stock"] == stock]["Date"],
                y=df[df["Stock"] == stock]["Volume"],
                mode='lines', 
                opacity=0.7,
                name=f'Volume {dropdown[stock]}', 
                textposition='bottom center'
            )
        )
    
    figure = {
        'data': trace1,
        'layout': go.Layout(
            colorway=["#5E0DAC", '#FF4F00', '#375CB1', '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"Market Volume for {', '.join(dropdown[i] for i in selected_dropdown_value)} Over Time",
            xaxis={
                "title": "Date",
                'rangeselector': {
                    'buttons': [
                        {'count': 1, 'label': '1M', 'step': 'month', 'stepmode': 'backward'},
                        {'count': 6, 'label': '6M', 'step': 'month', 'stepmode': 'backward'},
                        {'step': 'all'}
                    ]
                },
                'rangeslider': {'visible': True}, 
                'type': 'date'
            },
            yaxis={"title": "Transactions Volume"}
        )
    }
    return figure

if __name__ == '__main__':
    app.run_server(debug=True)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step
